In [23]:
import pandas as pd
import numpy as np
import re
import string
from IPython.display import display
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

import statsmodels.api as sm

In [2]:
PETR4_raw = pd.read_excel (r'PETR4.SA .xltx')

GOLD_raw = pd.read_excel (r'Gold Futures Historical Data.xltx') 
RDS_raw = pd.read_excel (r'RDS-A.xltx') 
OIL_raw = pd.read_excel (r'Crude Oil WTI Futures Historical Data.xltx')
CRD_raw = pd.read_excel (r'USD_BRL Dados Históricos.xltx')

In [3]:
from datetime import datetime
str_date = '06.11.2019'
def padronizaDia(str_date):
    date = datetime.strptime(str_date, '%d.%m.%Y').date()
    if len(str(date.day))==1:
        dia = "0" + str(date.day)
    else:
        dia = str(date.day)
        
    if len(str(date.month))==1:
        mes = "0" + str(date.month)
    else:
        mes = str(date.month)
    return "{0}-{1}-{2}".format(date.year,mes, dia)

In [4]:
PETR4 = PETR4_raw[['Date', 'Adj Close']]

GOLD = GOLD_raw[['Date', 'Price']]
RDS = RDS_raw[['Date', 'Adj Close']]
OIL = OIL_raw[['Date', 'Price']]
CRD = CRD_raw[['Data', 'Último']]


In [5]:
PETR4.head()

,Date,Adj Close
0,2014-11-05,13.634769
1,2014-11-06,13.312840
2,2014-11-07,13.511680
3,2014-11-10,13.237089
4,2014-11-11,13.256026


In [6]:
PETR4.set_index('Date', inplace = True)
GOLD.set_index('Date', inplace = True)
RDS.set_index('Date', inplace = True)
OIL.set_index('Date', inplace = True)


In [7]:
CRD.head()

,Data,Último
0,06.11.2019,40837
1,05.11.2019,39937
2,04.11.2019,40156
3,01.11.2019,39903
4,31.10.2019,40184


In [8]:
from datetime import datetime
str_date = '06.11.2019'
def padronizaDia(str_date):
    date = datetime.strptime(str_date, '%d.%m.%Y').date()
    if len(str(date.day))==1:
        dia = "0" + str(date.day)
    else:
        dia = str(date.day)
    return "{0}-{1}-{2}".format(date.year,date.month, dia)
    

In [9]:
CRD.head()

,Data,Último
0,06.11.2019,40837
1,05.11.2019,39937
2,04.11.2019,40156
3,01.11.2019,39903
4,31.10.2019,40184


In [10]:
CRD['Date'] = CRD.Data.apply(padronizaDia)
CRD.set_index('Date', inplace = True)

C:\Users\Tonera\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
df = PETR4.join(CRD, how='inner', lsuffix='_PETR4', rsuffix='_CRD')
df.head()

,Adj Close,Data,Último
Date,,,
2014-11-05,13.634769,05.11.2014,25049
2014-11-06,13.312840,06.11.2014,25697
2014-11-07,13.511680,07.11.2014,25587
2014-11-10,13.237089,10.11.2014,25528
2014-11-11,13.256026,11.11.2014,25575


In [14]:
df.drop(['Data'], axis=1, inplace = True)

In [15]:
df = df.join(GOLD, how = 'inner', rsuffix='_GOLD')

In [16]:
df = df.join(RDS, how = 'inner', rsuffix = '_RDS')

In [31]:
DataFrame = df.join(OIL, how = 'inner', rsuffix = '_OIL')
DataFrame.head()

,Adj Close,Data,Último,Price
Date,,,,
2014-11-05,13.634769,05.11.2014,25049,78.68
2014-11-06,13.312840,06.11.2014,25697,77.91
2014-11-07,13.511680,07.11.2014,25587,78.65
2014-11-10,13.237089,10.11.2014,25528,77.40
2014-11-11,13.256026,11.11.2014,25575,77.94


## Início da Regressão

In [18]:
### Função utilitária para fazer a regressão com constante adicionada
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

In [19]:
np.sum(DataFrame.isnull())

Adj Close        0
Último           0
Price            0
Adj Close_RDS    0
Price_OIL        0
dtype: int64

In [28]:
Y = DataFrame['Adj Close']
X = DataFrame[['Último', 'Price', 'Adj Close_RDS', 'Price_OIL']]

In [29]:
results = regress(X, Y)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Adj Close   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.786
Method:                 Least Squares   F-statistic:                     1113.
Date:                Mon, 11 Nov 2019   Prob (F-statistic):               0.00
Time:                        18:11:57   Log-Likelihood:                -3089.0
No. Observations:                1210   AIC:                             6188.
Df Residuals:                    1205   BIC:                             6213.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           -49.7938      1.889    -26.366      0.000     -53.499     -46.088
Último            0.0004   2.63e-05     16.138      0.000       0.000       0.000
Price             0.0205      0.001     15.869      0.000       0.018       0.023
Adj Close_RDS     0.4680      0.023     20.759      0.000       0.424       0.512
Price_OIL         0.0026      0.018      0.140      0.889      -0.033       0.039
==============================================================================
Omnibus:                       10.465   Durbin-Watson:                   0.029
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.599
Skew:                          -0.218   Prob(JB):                      0.00499
Kurtosis:                       2.857   Cond. No.                     7.38e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.38e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""